# Vetetation Index

This notebook takes the prepared images from "Crop Image" and the dataframe from "Dataframe_Mean_Green" to create four vegetation indicies for each image. The indicies used are Excess Green (ExG), Excess Green minus Red (ExGR), Green Leaf Index (GLI), and Visible Atmospherically Resistant Index (VARI). First Index directories are created (if they don't already exist). Then Each index is defined:

ExG = 2Green(n) - Red(n) - Blue(n)

ExGR = 3Green(n) - 2.4Red(n) - Blue(n)

GLI = 2Green(c) - Red(c) - Blue(c)/2Green(c) + Red(c) + Blue(c)

VARI = Green(c) - Red(c)/Green(c) + Red(c) - Blue(c)

Where Band(c) is the extracted band from the image ranging from 0-255 and Band(n) is a normalized version of that band defined as:
 
Band(n) = Band(c)/Red(c) + Green(c) + Blue(c)

The outputs collect the indicies to their respective outputs and the mean values of each image index.

A few notes: first, all the color bands were converted from dtype 'uint8' to a float so that the math would work. Second any invaid value (e.g. 0/0) was redefined as 0.

Luke Rosen

4/28/2022


Update 5/19/2022

Change "Mean" to "Avg".

In [1]:
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import skimage
from skimage.io import imread, imsave
from  datetime import datetime
import pandas as pd

In [2]:
#Opening and moving old dataframe.
Oat_Data = pd.read_csv("./Oat_Data.csv")
if not os.path.exists("./Archive"):
    os.mkdir("./Archive")
Oat_Data.to_csv("./Archive/Oat_Data_" + str(datetime.today().month) + "_" + str(datetime.today().day) + ".csv", index = False)



In [3]:
#Creating Directories
src = ("./images_prepared/")
exg_dir =  "./EXG/"
exgr_dir = "./EXGR/"
gli_dir = "./GLI/"
vari_dir = "./VARI/"
if not os.path.exists(exg_dir):
    os.mkdir(exg_dir)
if not os.path.exists(exgr_dir):
    os.mkdir(exgr_dir)
if not os.path.exists(gli_dir):
    os.mkdir(gli_dir)
if not os.path.exists(vari_dir):
    os.mkdir(vari_dir)


Defining each index

In [4]:
def EXG(output = None, name = None, Red_N= None, Green_N= None, Blue_N=None, EXG_Mean_Image = None): 
    result = (2.0 * Green_N) - Red_N - Blue_N  #ExG
    EXG_Mean_Image.append(result.mean())         #Mean Index
    imsave(output + name +"_EXG.tif", result) #appending mean index result

In [5]:
def EXGR(output = None, name = None, Red_N= None, Green_N= None, Blue_N=None, EXGR_Mean_Image = None):
    result = (3.0 * Green_N) - (2.4 * Red_N) - Blue_N  #ExGR
    EXGR_Mean_Image.append(result.mean())       #Mean Index  
    imsave(output + name + "_EXGR.tif", result) #appending mean index result

In [6]:
def GLI(output = None, name = None, Red_C= None, Green_C= None, Blue_C=None, GLI_Mean_Image = None):
    result = ((2.0 * Green_C) - Red_C - Blue_C)/((2.0 * Green_C) + Red_C + Blue_C) #GLI
    result[np.isnan(result)] = 0    #0/0 = 0
    result[np.isinf(result)] = 0    #N/0 = 0
    GLI_Mean_Image.append(result.mean()) #Mean Index
    imsave(output + name + "_GLI.tif", result) #appending mean index result

In [7]:
def VARI(output = None, name = None, Red_C= None, Green_C= None, Blue_C=None, VARI_Mean_Image = None):
    result = ((1.0 * Green_C) - Red_C)/((1.0 * Green_C) + Red_C - Blue_C)     #VARI
    result[np.isnan(result)] = 0      #0/0 = 0
    result[np.isinf(result)] = 0      #N/0 =0
    VARI_Mean_Image.append(result.mean())       #Mean Index
    imsave(output + name + "_VARI.tif", result) #appending mean index result

## Running the Function

This next cell uses the Oat Data dataframe to access the images and perform the indicies. First, empty colums are made to append the mean results. Then, the dataframe is used to open each image. Each image band is defined as well as the normailzed band. Finally the defined indicies are performed. The final cell saves the dataframe. 

In [8]:
count = 0
EXG_Mean_Image = []
EXGR_Mean_Image = []
GLI_Mean_Image = []
VARI_Mean_Image = []

with np.errstate(divide = "ignore", invalid = "ignore"):
    print("Starting Vegetation Indices...")
    for index, row in Oat_Data.iterrows():
        name = row.Image
        
        img = imread(src + name + ".tif")        
        
        Red_c = img[:,:,0] * 1.0     ## Red color band as float

        Green_c = img[:,:,1] * 1.0   ## Green color band as float
 
        Blue_c = img[:,:,2] * 1.0    ## Blue color band as fload

        Red_n = Red_c/(Red_c + Green_c + Blue_c)   ## Normalize Red
        Red_n[np.isnan(Red_n)] = 0
        Red_n[np.isinf(Red_n)] = 0

        Green_n = Green_c/(Red_c + Green_c + Blue_c) ## Normalize Green
        Green_n[np.isnan(Green_n)] = 0
        Green_n[np.isinf(Green_n)] = 0

        Blue_n = Blue_c/(Red_c + Green_c + Blue_c) ## Normalize Blue
        Blue_n[np.isnan(Blue_n)] = 0
        Blue_n[np.isinf(Blue_n)] = 0
        
        EXG(exg_dir, name, Red_n, Green_n, Blue_n, EXG_Mean_Image)
        EXGR(exgr_dir, name, Red_n, Green_n, Blue_n, EXGR_Mean_Image)
        GLI(gli_dir, name, Red_c, Green_c, Blue_c, GLI_Mean_Image)
        VARI(vari_dir, name, Red_c, Green_c, Blue_c, VARI_Mean_Image)
        if count % 42 == 0:
            print("Done with image " + name + "...\n")
        count = count + 1
    print("All images done...\n")
print("Vegetaion Index Complete!!!")

Starting Vegetation Indices...
Done with image 101A_20210708...

Done with image 205A_20210708...

Done with image 309A_20210708...

Done with image 103A_20210821...

Done with image 207A_20210821...

Done with image 401A_20210821...

Done with image 105A_20210909...

Done with image 209A_20210909...

Done with image 403A_20210909...

Done with image 107A_20210929...

Done with image 301A_20210929...

Done with image 405A_20210929...

Done with image 109A_20211005...

Done with image 303A_20211005...

Done with image 407A_20211005...

Done with image 201A_20211015...

Done with image 305A_20211015...

Done with image 409A_20211015...

Done with image 203A_20211025...

Done with image 307A_20211025...

All images done...

Vegetaion Index Complete!!!


In [9]:
#Appending and saving dataframe.
Oat_Data["EXG_Avg_Image"] = EXG_Mean_Image
Oat_Data["EXGR_Avg_Image"] = EXGR_Mean_Image
Oat_Data["GLI_Avg_Image"] = GLI_Mean_Image
Oat_Data["VARI_Avg_Image"] = VARI_Mean_Image

Oat_Data.to_csv("./Oat_Data.csv", index = False)

This notebook is complete. The next notebook will use Otsu's Threshold locally to classifiy the indexed images. 